In [ ]:
import numpy as np

In [ ]:
def prod_0(tabx, taby):
    res =[]
    for a,b in zip(tabx, taby):
        res.append(a * b)
    return np.array(res)

In [ ]:
def prod_1(tabx, taby):
    return np.array([a * b for a,b in zip(tabx, taby)])

In [ ]:
import random as rd
l_A = [rd.random() for i in range(1000000)]
l_B = [rd.random() for i in range(1000000)]
%time prod_0(l_A, l_B)

In [ ]:
%timeit prod_0(l_A, l_B)

# profiling

In [ ]:
import cProfile

In [ ]:
from fibo import fib_rec as fib

In [ ]:
cProfile.run('fib(30)')

## `line_profiler`
* https://github.com/pyutils/line_profiler

In [ ]:
%load_ext line_profiler

In [ ]:
%lprun -f prod_0 prod_0(l_A,l_B)

## `memory_profiler`
* https://github.com/pythonprofilers/memory_profiler

In [ ]:
%load_ext memory_profiler

In [ ]:
import test_mp

In [ ]:
%mprun -f test_mp.my_func test_mp.my_func()

# Slow Python

## C

In [ ]:
!gcc -O3 -std=gnu99 vbyv.c

In [ ]:
!./a.out 1000000

## Pure Python

In [ ]:
%timeit prod_0(l_A, l_B)
%timeit prod_1(l_A, l_B)

## Numpy

In [ ]:
v_A = np.array(l_A)
v_B = np.array(l_B)
%timeit v_A * v_B

In [ ]:
%timeit prod_0(v_A, v_B)
%timeit prod_1(v_A, v_B)

## Another example

In [ ]:
!gcc -O3 -std=gnu99 integrate_f_c.c -o integrate_c.exe

In [ ]:
!./integrate_c.exe -1 1 10000

In [ ]:
from integrate_f_py import integrate_f_py as py_integrate

In [ ]:
%timeit py_integrate(-1,1,10000)

---
# compilation

---
## Cython
<img src="images/cython.png" alt="Cython" width="200"/>

* https://cython.org/

In [ ]:
from integrate_f_py import integrate_f_py as py_integrate

In [ ]:
%timeit py_integrate(-1,1,10000)

In [ ]:
!cp integrate_f_py.py integrate_f_cy.pyx

In [ ]:
!cythonize -b -i integrate_f_cy.pyx

In [ ]:
!ls 

In [ ]:
!cp build/lib.macosx-10.9-x86_64-cpython-310/integrate_f_cy.cpython-310-darwin.so ./

In [ ]:
from integrate_f_cy import integrate_f_py as cy_integrate

In [ ]:
ls exemple-cython/

---
## Numba
<img src="images/numba.png" alt="Numba" width="200"/>

In [ ]:
def f(x):
    return -4 * x**3 + 3 * x**2 + 2*x

def integrate_f_py(a, b, N):
    s  = 0
    dx = (b - a) / (N - 1)
    # mauvaise boucle
    for i in range(N - 1): 
        x = a + i * dx
        s += (f(x) + f(x + dx)) / 2
    return s * dx

In [ ]:
%timeit integrate_f_py(-1,1,10000)

In [ ]:
from numba import jit,njit

In [ ]:
@njit
def f_jit(x):
    return -4 * x**3 + 3 * x**2 + 2*x

In [ ]:
def integrate_f_jit(a, b, N):
    s  = 0
    dx = (b - a) / (N - 1)
    # mauvaise boucle
    for i in range(N - 1): 
        x = a + i * dx
        s += (f_jit(x) + f_jit(x + dx)) / 2
    return s * dx

In [ ]:
integrate_f_jit(-1,1,10000)

In [ ]:
%timeit integrate_f_jit(-1,1,10000)

In [ ]:
@njit
def integrate_f_jit(a, b, N):
    s  = 0
    dx = (b - a) / (N - 1)
    # mauvaise boucle
    for i in range(N - 1): 
        x = a + i * dx
        s += (f_jit(x) + f_jit(x + dx)) / 2
    return s * dx

In [ ]:
integrate_f_jit(-1,1,10000)

In [ ]:
%timeit integrate_f_jit(-1,1,10000)

In [ ]:
%timeit integrate_f_jit.py_func(-1,1,10000)

---
# Multithreading
<img src="images/multithread.png" alt="multithreading" width="200"/>

In [ ]:
def create_list(n):
    res = []
    for i in range(n):
        res.append(i)
    return res

In [ ]:
size = 100000
%timeit create_list(size)

In [ ]:
from concurrent.futures import ThreadPoolExecutor

In [ ]:
t_exe = ThreadPoolExecutor(4)

In [ ]:
lt = [res for res in t_exe.map(create_list, [size, size])]

In [ ]:
lt = [res for res in t_exe.map(create_list, [size, size])]

In [ ]:
%time lt = [res for res in t_exe.map(create_list, [size, size])]

In [ ]:
%time lnt = [res for res in map(create_list, [size, size])]

In [ ]:
import time
def attendre(t=0.009):
    time.sleep(t)
    return None

In [ ]:
%time attendre(10)

In [ ]:
%time for res in t_exe.map(attendre,[10, 10]) : pass

In [ ]:
import math
def is_prime(n):
    if n < 2:
        return False
    sqrt_n = int(math.floor(math.sqrt(n)))
    for i in range(2, sqrt_n + 1):
        if n % i == 0:
            return False
    return True

In [ ]:
primes = [112272535095293 + i for i in range(0,1000)]

In [ ]:
%time is_prime_jit(112272535095293)

In [ ]:
%time l0 = [res for res in map(is_prime,primes[0:100])]

In [ ]:
%time lt = [res for res in t_exe.map(is_prime, primes[0:100])]

## Numba nogil

In [ ]:
@njit
def is_prime_jit(n):
    if n < 2:
        return False
    sqrt_n = int(math.floor(math.sqrt(n)))
    for i in range(2, sqrt_n + 1):
        if n % i == 0:
            return False
    return True

In [ ]:
is_prime_jit(3)

In [ ]:
%time lnt = [res for res in map(is_prime_jit,primes[0:1000])]

In [ ]:
%time lnt = [res for res in t_exe.map(is_prime_jit, primes[0:1000])]

In [ ]:
@njit(nogil = True)
def is_prime_nogil(n):
    if n < 2:
        return False
    sqrt_n = int(math.floor(math.sqrt(n)))
    for i in range(2, sqrt_n + 1):
        if n % i == 0:
            return False
    return True

In [ ]:
is_prime_nogil(3)

In [ ]:
%time ltng = [res for res in t_exe.map(is_prime_nogil, primes[0:1000])]